In [1]:
import sqlite3 as sql
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pandas_datareader import data as web 


import os
notebook_directory = os.getcwd()
print(notebook_directory)


/Users/lime/src/data-analytics/WebApp_Dashboard


In [2]:




def load_data():
    # Load the data from SQL database
    conn = sql.connect('/Users/lime/src/data-analytics/data/Crypto.db') 
    df_sum = pd.read_sql_query("SELECT * FROM PORTFOLIO", conn)
    conn.close

    # Data preparation
    df_sum['TotalAmount'] = df_sum.groupby('Ticker')['Amount'].transform('sum')
    df_sum.rename(columns={'TotalAmount':'Quantity'}, inplace=True)

    pd.set_option('display.float_format', '{:.6f}'.format)
    df_prepeared = df_sum[['Ticker', 'Quantity']].drop_duplicates()
    tickers_to_drop = ['VTX', 'CITY', 'IONX']# Drop a few rows for now due to an issue with the market maker. In the future, try using API requests with another financial aggregator.
    df_prepeared = df_prepeared[~df_prepeared['Ticker'].isin(tickers_to_drop)]

    return df_prepeared, df_sum

In [3]:
df_prepeared, df_sum = load_data()

In [4]:
df_prepeared = df_prepeared.drop("Quantity", axis=1)
df_prepeared = df_prepeared.drop_duplicates(subset='Ticker')
df_prepeared = df_prepeared.reset_index(drop=True)

print(df_prepeared)


   Ticker
0     TWT
1     BTC
2     ETH
3     BNB
4     JOE
5    MINA
6   MATIC
7    AVAX
8    ATOM
9    DYDX
10    SOL
11  1INCH
12    TON
13    DOT
14    KSM
15   PERP
16   KAVA
17    ILV
18   ASTR
19    GRT
20   ALCX
21    UNI
22    RVN
23    APT
24    XRP
25   ANKR
26    TRX
27    CVX
28    LTO
29    ENJ
30  OCEAN
31   BICO
32    ONE
33  SUSHI
34   DOGE
35   MANA
36    EOS
37   SHIB
38   SAND
39    MOB
40    CRV
41  PORTO
42   LUNA
43   LUNC
44    XVG
45    KAR
46   TOKE
47   SAKE
48    IMX


In [5]:
df_prepeared = df_prepeared.reset_index(drop=True)
print(df_prepeared)

   Ticker
0     TWT
1     BTC
2     ETH
3     BNB
4     JOE
5    MINA
6   MATIC
7    AVAX
8    ATOM
9    DYDX
10    SOL
11  1INCH
12    TON
13    DOT
14    KSM
15   PERP
16   KAVA
17    ILV
18   ASTR
19    GRT
20   ALCX
21    UNI
22    RVN
23    APT
24    XRP
25   ANKR
26    TRX
27    CVX
28    LTO
29    ENJ
30  OCEAN
31   BICO
32    ONE
33  SUSHI
34   DOGE
35   MANA
36    EOS
37   SHIB
38   SAND
39    MOB
40    CRV
41  PORTO
42   LUNA
43   LUNC
44    XVG
45    KAR
46   TOKE
47   SAKE
48    IMX


In [6]:
def insert_data(conn, df_prepeared):
    conn.execute('''INSERT INTO TOKENSINFO (Ticker, Information) VALUES (?, ?)''', (df_prepeared, ''))
    conn.commit()



In [7]:
conn = sql.connect('/Users/lime/src/data-analytics/data/Crypto.db')


for index, row in df_prepeared.iterrows():
    ticker = row['Ticker']
    insert_data(conn, ticker)
conn.commit()
conn.close()